# Differential equation in sagemath

In [1]:
# ----------------------------
# 0) Setup
# ----------------------------
var('t w', domain='real')
y = function('y')(t)

# initial conditions (edit v0 if desired)
y0  = 1
v0  = 0           # y'(0)
T   = 100.0       # integration horizon for numerics
h   = 0.001       # step size for RK4
omegas = [1, 10, 100]  # test set of frequencies

# simple timing helpers (CPU + wall)
def bench(fun, n=3, label=''):
    import time
    from sage.misc.misc import cputime, walltime
    # warmup
    fun()
    c0, w0 = cputime(), walltime()
    for _ in range(n):
        fun()
    c1, w1 = cputime(c0), walltime(w0)
    print(f"[{label}] runs={n}  cpu={c1:.4f}s  wall={w1:.4f}s  avg_wall={w1/n:.4f}s")

# ----------------------------
# 1) Symbolic solution (desolve)
#     y'' + w^2 y = 0,  y(0)=y0, y'(0)=v0
# ----------------------------
ode = diff(y,t,2) + w^2*y == 0
sol = desolve(ode, y, ics=[0, y0, v0]).simplify_full()
show(sol)
# Expected: y(t) = y0*cos(w*t) + (v0/w)*sin(w*t)

# Optional: make a fast numeric callable for evaluation
y_sym = sol.rhs()
y_sym_num = fast_callable(y_sym.subs({w:RR(10)}), vars=[t])  # example with w=10
print(y_sym_num(0.123))

# ----------------------------
# 2) Numeric RK4 (first-order system form)
#     y1 = y, y2 = y'
#     y1' = y2
#     y2' = -w^2 * y1
# ----------------------------
def rk4_trajectory(wval, T=10.0, h=0.001, y0=1.0, v0=0.0):
    y1, y2 = function('y1')(t), function('y2')(t)
    F = [y2, -wval^2 * y1]
    pts = desolve_system_rk4(F, [y1, y2], ics=[0, y0, v0],
                             ivar=t, end_points=T, step=h)
    return pts  # list of [t, y1, y2]

# quick correctness check vs symbolic at w=10
wtest = 10
pts = rk4_trajectory(wtest, T=2.0, h=0.0005, y0=y0, v0=v0)
max_err = max(abs(p[1] - y_sym.subs({w:wtest, t:p[0]}).n()) for p in pts)
print(f"max |y_RK4 - y_sym|, w={wtest}: {max_err}")

# ----------------------------
# 3) Benchmarks
#    Compare:
#      A) desolve (symbolic)
#      B) evaluating closed-form fast_callable on a dense grid
#      C) RK4 integration
# ----------------------------
def bench_symbolic_desolve():
    # solve once per omega (fresh symbol each time to be fair)
    for wval in omegas:
        _ = desolve(diff(function('Y')(t),t,2) + wval^2*function('Y')(t) == 0,
                    function('Y')(t), ics=[0, y0, v0])

def bench_closed_form_eval():
    ts = srange(0, T, h)
    for wval in omegas:
        expr = (y0*cos(wval*t) + (v0/wval)*sin(wval*t)).simplify_full() if wval != 0 else y0
        f = fast_callable(expr, vars=[t])
        s = 0.0
        for tt in ts:
            s += f(tt)    # force evaluation loop

def bench_rk4():
    for wval in omegas:
        _ = rk4_trajectory(wval, T=T, h=h, y0=y0, v0=v0)

print("\n--- BENCHMARKS ---")
bench(bench_symbolic_desolve, n=5, label='desolve (solve once per ω)')
bench(bench_closed_form_eval, n=3, label='closed-form eval on grid')
bench(bench_rk4, n=3, label='RK4 on grid')

# ----------------------------
# 4) Optional: plots for one ω
# ----------------------------
wplot = 10
sol_plot = (y0*cos(wplot*t) + (v0/wplot)*sin(wplot*t)).simplify_full()
p_sym = plot(sol_plot, (t, 0, 2*pi), legend_label='symbolic', ymin=-1.2, ymax=1.2)
pts_num = rk4_trajectory(wplot, T=2*pi, h=0.001, y0=y0, v0=v0)
p_num = line([(p[0], p[1]) for p in pts_num], color='red', legend_label='RK4')
(p_sym + p_num).show()


ValueError: Unable to determine independent variable, please specify.